In [1]:
!pip install imbalanced-learn

In [2]:
pip install shap imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Expanded imports for ROC curves and cross-validation
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import (classification_report, confusion_matrix, ConfusionMatrixDisplay, 
                           roc_curve, auc, roc_auc_score)
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE
from scipy.sparse import hstack, csr_matrix
import shap
import os

# Ensure output directory exists
os.makedirs("cleaned_data/final/imgs", exist_ok=True)

# Load dataset
df = pd.read_csv("cleaned_data/final/BERT/all_merged_labeled_contextual.csv")

# 1. Process time-based features (new addition)
print("Processing time features...")
# Convert event_time to datetime type
df['event_time'] = pd.to_datetime(df['event_time'], errors='coerce')

# Extract time features
time_features = pd.DataFrame(index=df.index)
# Only extract features for non-null dates
valid_dates = df['event_time'].notna()
if valid_dates.any():
    time_features.loc[valid_dates, 'hour'] = df.loc[valid_dates, 'event_time'].dt.hour
    time_features.loc[valid_dates, 'day_of_week'] = df.loc[valid_dates, 'event_time'].dt.dayofweek
    time_features.loc[valid_dates, 'month'] = df.loc[valid_dates, 'event_time'].dt.month
    time_features.loc[valid_dates, 'is_weekend'] = df.loc[valid_dates, 'event_time'].dt.dayofweek >= 5
    
    # Create day_night feature (7am-7pm is day)
    valid_indices = df.index[valid_dates]
    day_hour_mask = (df.loc[valid_dates, 'event_time'].dt.hour >= 7) & (df.loc[valid_dates, 'event_time'].dt.hour < 19)
    day_indices = valid_indices[day_hour_mask]
    night_indices = valid_indices[~day_hour_mask]

    time_features.loc[day_indices, 'day_night'] = 'day'
    time_features.loc[night_indices, 'day_night'] = 'night'
    
    # Fill NaN values with appropriate default values
    time_features = time_features.fillna(-1)  # -1 represents unknown time
    
    # Visualize event distribution across different time periods
    plt.figure(figsize=(12, 6))
    
    # Plot events by hour of day
    plt.subplot(1, 2, 1)
    hour_counts = time_features.loc[valid_dates, 'hour'].value_counts().sort_index()
    sns.barplot(x=hour_counts.index, y=hour_counts.values)
    plt.title('Events by Hour of Day')
    plt.xlabel('Hour')
    plt.ylabel('Event Count')
    
    # Plot events by day of week
    plt.subplot(1, 2, 2)
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_counts = time_features.loc[valid_dates, 'day_of_week'].value_counts().sort_index()

    # Convert float indices to integers before using them as list indices
    sns.barplot(x=[days[int(i)] for i in day_counts.index], y=day_counts.values)
    plt.title('Events by Day of Week')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig("cleaned_data/final/imgs/time_feature_distribution.png")
    plt.close()

    print(f"Successfully extracted time features for {valid_dates.sum()} records")
else:
    print("Warning: No valid time data available")

# One-hot encode categorical variables
if 'day_night' in time_features.columns:
    time_features = pd.get_dummies(time_features, columns=['day_night'])

# Extended EDA Analysis
# Add this code section after the time feature extraction but before model training

print("\n\nPerforming Extended EDA Analysis...")

# Create a directory for additional EDA visualizations
os.makedirs("cleaned_data/final/imgs/extended_eda", exist_ok=True)

# -----------------------------------------------------------------------
# 1. Night Shift/Day Shift Violence Analysis by Role
# -----------------------------------------------------------------------
print("Analyzing violence patterns by shift and profession...")

# Create a combined dataframe with time features for analysis
if 'day_night' in time_features.columns and valid_dates.any():
    # We need to recreate day_night as a string column (not one-hot encoded)
    role_time_df = df.join(time_features)
    
    # Get top 5 most common victim professions for readability
    top_professions = df['victim_profession'].value_counts().nlargest(5).index.tolist()
    
    # Filter for only top professions and valid time data
    role_time_filtered = role_time_df[
        role_time_df['victim_profession'].isin(top_professions) & 
        valid_dates
    ]
    
    # Plot count by profession and day/night
    plt.figure(figsize=(12, 8))
    
    # First subplot: absolute counts
    plt.subplot(2, 1, 1)
    shift_count = sns.countplot(data=role_time_filtered, 
                               x='victim_profession', 
                               hue='day_night')
    plt.title('Incident Counts by Profession and Shift')
    plt.xlabel('Victim Profession')
    plt.ylabel('Number of Incidents')
    plt.xticks(rotation=45)
    
    # Second subplot: proportions within each profession
    plt.subplot(2, 1, 2)
    
    # Calculate proportions of day vs night incidents for each profession
    prof_shift_props = pd.crosstab(
        role_time_filtered['victim_profession'], 
        role_time_filtered['day_night'],
        normalize='index'
    )
    
    # Plot as stacked bars
    prof_shift_props.plot(kind='bar', stacked=True, figsize=(12, 6))
    plt.title('Proportion of Day vs Night Incidents by Profession')
    plt.xlabel('Victim Profession')
    plt.ylabel('Proportion of Incidents')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig("cleaned_data/final/imgs/extended_eda/profession_by_shift.png")
    plt.close()
    
    # Additional analysis: Calculate incident rate per shift by profession
    print("\nDay vs Night Shift Incident Rates by Profession:")
    shift_rates = pd.crosstab(role_time_filtered['victim_profession'], 
                             role_time_filtered['day_night'])
    
    # Calculate ratio of night to day incidents (higher = more night incidents)
    if 'night' in shift_rates.columns and 'day' in shift_rates.columns:
        shift_rates['night_day_ratio'] = shift_rates['night'] / shift_rates['day']
        print(shift_rates.sort_values('night_day_ratio', ascending=False))
        
        # Plot the night/day ratio
        plt.figure(figsize=(10, 6))
        sns.barplot(x=shift_rates.index, y=shift_rates['night_day_ratio'])
        plt.axhline(y=1.0, color='r', linestyle='--', alpha=0.7)  # Reference line where night=day
        plt.title('Night to Day Incident Ratio by Profession')
        plt.xlabel('Victim Profession')
        plt.ylabel('Night/Day Incident Ratio')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig("cleaned_data/final/imgs/extended_eda/night_day_ratio.png")
        plt.close()

# -----------------------------------------------------------------------
# 2. Violence Type and Emotional Impact Analysis
# -----------------------------------------------------------------------
print("Analyzing relationship between violence type and emotional impact...")

# Check if we have the necessary columns
if all(col in df.columns for col in ['violence_type', 'emotional_impact_ML']):
    # Create cross-tabulation
    violence_impact = pd.crosstab(
        df['violence_type'], 
        df['emotional_impact_ML'],
        normalize='index'  # Normalize by row to get proportions within each violence type
    )
    
    # Plot heatmap
    plt.figure(figsize=(10, 6))
    sns.heatmap(violence_impact, annot=True, cmap='YlOrRd', fmt='.2%')
    plt.title('Emotional Impact by Violence Type')
    plt.ylabel('Violence Type')
    plt.xlabel('Emotional Impact')
    plt.tight_layout()
    plt.savefig("cleaned_data/final/imgs/extended_eda/violence_emotional_impact.png")
    plt.close()
    
    # Plot stacked bar chart
    plt.figure(figsize=(10, 6))
    violence_impact.plot(kind='bar', stacked=True)
    plt.title('Distribution of Emotional Impact by Violence Type')
    plt.xlabel('Violence Type')
    plt.ylabel('Proportion')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig("cleaned_data/final/imgs/extended_eda/violence_impact_bars.png")
    plt.close()
    
    # If 'days_missed' or similar column exists, create boxplot
    if 'days_missed' in df.columns:
        plt.figure(figsize=(10, 6))
        sns.boxplot(data=df, x='violence_type', y='days_missed')
        plt.title('Days Missed by Violence Type')
        plt.xlabel('Violence Type')
        plt.ylabel('Days Missed')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig("cleaned_data/final/imgs/extended_eda/violence_days_missed.png")
        plt.close()

# -----------------------------------------------------------------------
# 3. Text Embedding Visualization using t-SNE
# -----------------------------------------------------------------------
print("Generating text embedding visualization...")

# We've already created TF-IDF vectors for the assault descriptions
from sklearn.manifold import TSNE

# Sample a subset of the text data to make visualization manageable
max_samples = min(500, X_text.shape[0])  # Use at most 500 samples
indices = np.random.choice(X_text.shape[0], max_samples, replace=False)

# Get the subset of text vectors and corresponding labels
X_text_sample = X_text[indices]
y_sample = df.iloc[indices]['emotional_impact_ML'].values

# Apply t-SNE to reduce to 2D
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_text_tsne = tsne.fit_transform(X_text_sample.toarray())

# Create a DataFrame for plotting
tsne_df = pd.DataFrame({
    'x': X_text_tsne[:, 0],
    'y': X_text_tsne[:, 1],
    'impact': y_sample,
    'violence_type': df.iloc[indices]['violence_type'].values if 'violence_type' in df.columns else ['Unknown'] * max_samples
})

# Plot by emotional impact
plt.figure(figsize=(12, 10))
plt.subplot(2, 1, 1)
sns.scatterplot(data=tsne_df, x='x', y='y', hue='impact', palette='viridis', s=50, alpha=0.7)
plt.title('t-SNE Visualization of Assault Descriptions by Emotional Impact')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend(title='Emotional Impact')

# Plot by violence type
plt.subplot(2, 1, 2)
sns.scatterplot(data=tsne_df, x='x', y='y', hue='violence_type', palette='Set2', s=50, alpha=0.7)
plt.title('t-SNE Visualization of Assault Descriptions by Violence Type')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend(title='Violence Type')

plt.tight_layout()
plt.savefig("cleaned_data/final/imgs/extended_eda/text_embedding_tsne.png")
plt.close()

# -----------------------------------------------------------------------
# 4. Department and Time Analysis
# -----------------------------------------------------------------------
print("Analyzing departmental patterns by time...")

if 'department' in df.columns and 'hour' in time_features.columns and valid_dates.any():
    # Get top departments by incident count
    top_depts = df['department'].value_counts().nlargest(5).index.tolist()
    
    # Create a dataframe with both department and hour information
    dept_time_df = df.join(time_features).dropna(subset=['department', 'hour'])
    dept_time_filtered = dept_time_df[dept_time_df['department'].isin(top_depts)]
    
    # Plot incident distribution by hour for each department
    plt.figure(figsize=(14, 8))
    
    # Group by department and hour, count incidents
    dept_hour_counts = dept_time_filtered.groupby(['department', 'hour']).size().reset_index(name='count')
    
    # Plot line chart
    sns.lineplot(data=dept_hour_counts, x='hour', y='count', hue='department', marker='o')
    plt.title('Incident Counts by Hour and Department')
    plt.xlabel('Hour of Day')
    plt.ylabel('Number of Incidents')
    plt.xticks(range(0, 24, 2))  # Show every other hour for readability
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig("cleaned_data/final/imgs/extended_eda/department_hour_distribution.png")
    plt.close()
    
    # Create a heatmap of departments by hour
    dept_hour_pivot = dept_hour_counts.pivot(index='department', columns='hour', values='count')
    dept_hour_pivot = dept_hour_pivot.fillna(0)
    
    plt.figure(figsize=(14, 8))
    sns.heatmap(dept_hour_pivot, cmap='YlOrRd', annot=True, fmt='.0f')
    plt.title('Incident Heatmap by Department and Hour')
    plt.xlabel('Hour of Day')
    plt.ylabel('Department')
    plt.tight_layout()
    plt.savefig("cleaned_data/final/imgs/extended_eda/department_hour_heatmap.png")
    plt.close()

# -----------------------------------------------------------------------
# 5. Time, Violence Type, and Severity Analysis
# -----------------------------------------------------------------------
print("Analyzing violence patterns by time of day...")

if all(col in df.columns for col in ['violence_type', 'emotional_impact_ML']) and 'hour' in time_features.columns:
    # Create a combined dataframe
    time_violence_df = df.join(time_features).dropna(subset=['violence_type', 'emotional_impact_ML', 'hour'])
    
    # Group by hour and violence type, count incidents
    hour_violence_counts = time_violence_df.groupby(['hour', 'violence_type']).size().reset_index(name='count')
    
    # Plot line chart
    plt.figure(figsize=(14, 8))
    sns.lineplot(data=hour_violence_counts, x='hour', y='count', hue='violence_type', marker='o')
    plt.title('Violence Type by Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('Number of Incidents')
    plt.xticks(range(0, 24, 2))
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig("cleaned_data/final/imgs/extended_eda/violence_type_hour.png")
    plt.close()
    
    # Create a heatmap of emotional impact severity by hour
    # Calculate proportion of severe incidents by hour
    severe_by_hour = time_violence_df.groupby('hour')['emotional_impact_ML'].apply(
        lambda x: (x == 'Severe').mean()
    ).reset_index(name='severe_proportion')
    
    plt.figure(figsize=(14, 6))
    sns.barplot(data=severe_by_hour, x='hour', y='severe_proportion')
    plt.title('Proportion of Severe Emotional Impact by Hour')
    plt.xlabel('Hour of Day')
    plt.ylabel('Proportion of Severe Cases')
    plt.xticks(range(0, 24))
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig("cleaned_data/final/imgs/extended_eda/severe_impact_by_hour.png")
    plt.close()

print("Extended EDA analysis complete!")

# Continue with standard data preprocessing
print("Preprocessing data...")
target = 'emotional_impact_ML'
text_col = 'assault_desc'
struct_features = ['victim_profession', 'department', 'perpetrator_type', 'violence_type', 'emotional_impact', 'response_action']
df = df.dropna(subset=struct_features + [target, text_col])

# Encode target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df[target])
target_names = label_encoder.classes_
print(f"Target classes: {target_names}")

# Structured feature encoding
X_struct = pd.get_dummies(df[struct_features])

# Text feature processing
tfidf = TfidfVectorizer(max_features=300, stop_words='english')
X_text = tfidf.fit_transform(df[text_col])
print(f"TF-IDF extracted {X_text.shape[1]} text features")

# 2. Combine all features (improved feature combination approach)
print("Combining features...")
feature_matrices = [X_struct]
feature_names = list(X_struct.columns)

# Add text features
feature_matrices.append(X_text)
feature_names.extend(tfidf.get_feature_names_out())



# Fix for converting time features to sparse matrix
if 'hour' in time_features.columns:
    # Get numeric columns only
    numeric_cols = time_features.select_dtypes(include=[np.number]).columns
    categorical_cols = time_features.select_dtypes(exclude=[np.number]).columns
    
    # Handle numeric features
    numeric_features = time_features[numeric_cols].reindex(df.index, fill_value=0)
    feature_matrices.append(csr_matrix(numeric_features.values))
    feature_names.extend(numeric_cols)
    
    # Handle categorical features - we need to one-hot encode these
    for cat_col in categorical_cols:
        # Skip if already one-hot encoded
        if cat_col != 'day_night':  # 'day_night' should already be one-hot encoded before this point
            cat_dummies = pd.get_dummies(time_features[cat_col], prefix=cat_col)
            cat_dummies_aligned = cat_dummies.reindex(df.index, fill_value=0)
            feature_matrices.append(csr_matrix(cat_dummies_aligned.values))
            feature_names.extend(cat_dummies_aligned.columns)
    
    print(f"Added time features to the model")
# Merge all features
X_combined = hstack(feature_matrices)
X_combined = StandardScaler(with_mean=False).fit_transform(X_combined)
print(f"Final feature matrix shape: {X_combined.shape}")

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.2, random_state=42, stratify=y
)

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print(f"Training set shape after SMOTE: {X_train_res.shape}")

# 3. Enhanced model training (added class weights and cross-validation)
print("Training and evaluating models...")
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, class_weight='balanced'),
    "Random Forest": RandomForestClassifier(class_weight='balanced', n_estimators=100),
    "SVM": SVC(probability=True, class_weight='balanced')
}

# Evaluate and save confusion matrices and ROC curves
model_scores = {}
labels = sorted(df[target].unique())

for name, model in models.items():
    print(f"\nTraining {name} model...")
    
    # Evaluate model performance using cross-validation
    cv_scores = cross_val_score(model, X_train_res, y_train_res, cv=5, scoring='f1_weighted')
    print(f"{name} - Cross-Validation Score (F1 weighted): {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    
    # Train on the full training set
    model.fit(X_train_res, y_train_res)
    y_pred = model.predict(X_test)
    
    # Get classification report
    report = classification_report(y_test, y_pred, target_names=labels, output_dict=True)
    model_scores[name] = report["weighted avg"]
    
    print(f"\n{name} - Classification Report")
    print(classification_report(y_test, y_pred, target_names=labels))
    
    # Plot confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    plt.figure(figsize=(8, 6))
    disp.plot(cmap="Blues")
    plt.title(f"{name} - Emotional Impact Classification")
    plt.savefig(f"cleaned_data/final/imgs/{name.replace(' ', '_').lower()}_cm.png", bbox_inches='tight')
    plt.close()
    
    # 4. Calculate and plot ROC curves (new addition)
    if hasattr(model, "predict_proba"):
        print(f"Generating ROC curves for {name}...")
        # Get prediction probabilities
        y_prob = model.predict_proba(X_test)
        
        # Calculate ROC and AUC for multiclass
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        
        # Prepare one-vs-rest classifier to calculate ROC for each class
        plt.figure(figsize=(8, 6))
        
        n_classes = len(labels)
        for i in range(n_classes):
            # Use current class as positive, others as negative
            y_test_binary = (y_test == i).astype(int)
            y_score = y_prob[:, i]
            
            fpr[i], tpr[i], _ = roc_curve(y_test_binary, y_score)
            roc_auc[i] = auc(fpr[i], tpr[i])
            
            plt.plot(fpr[i], tpr[i], lw=2,
                    label=f'{labels[i]} (AUC = {roc_auc[i]:.2f})')
        
        # Calculate average ROC curve
        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(n_classes):
            mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
        mean_tpr /= n_classes
        
        mean_auc = auc(all_fpr, mean_tpr)
        plt.plot(all_fpr, mean_tpr, 'k--',
                label=f'Average ROC (AUC = {mean_auc:.2f})', lw=2)
        
        plt.plot([0, 1], [0, 1], 'r--', lw=2)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve - {name}')
        plt.legend(loc="lower right")
        plt.savefig(f"cleaned_data/final/imgs/{name.replace(' ', '_').lower()}_roc.png", bbox_inches='tight')
        plt.close()

# 5. Enhanced feature importance analysis
print("Analyzing feature importance...")
if "Random Forest" in models:
    rf_model = models["Random Forest"]
    importances = rf_model.feature_importances_
    
    # Expanded to top 15
    indices = np.argsort(importances)[-15:][::-1]  
    
    plt.figure(figsize=(12, 8))
    sns.barplot(x=importances[indices], y=np.array(feature_names)[indices])
    plt.title("Top 15 Feature Importances (Random Forest)")
    plt.tight_layout()
    plt.savefig("cleaned_data/final/imgs/rf_feature_importance_extended.png")
    plt.close()
    
    # Analyze importance of time features
    time_feature_indices = [i for i, name in enumerate(feature_names) 
                           if any(time_prefix in name for time_prefix in 
                                 ['hour', 'day_of_week', 'month', 'is_weekend', 'day_night'])]
    
    if time_feature_indices:
        time_importances = importances[time_feature_indices]
        time_names = np.array(feature_names)[time_feature_indices]
        
        # Sort by importance
        time_sorted_idx = np.argsort(time_importances)[::-1]
        
        plt.figure(figsize=(10, 6))
        sns.barplot(x=time_importances[time_sorted_idx], y=time_names[time_sorted_idx])
        plt.title("Time-Based Feature Importance")
        plt.tight_layout()
        plt.savefig("cleaned_data/final/imgs/time_feature_importance.png")
        plt.close()
        print("Generated time feature importance plot")

# SHAP analysis (enhanced)
print("Performing SHAP analysis...")
X_train_sample = X_train_res[:500].toarray()  # Use larger sample for better explainability
X_test_sample = X_test[:100].toarray()

explainer = shap.Explainer(rf_model, X_train_sample)
shap_values = explainer(X_test_sample)

# Basic SHAP summary plot
shap.summary_plot(shap_values, X_test_sample, feature_names=feature_names, max_display=15, show=False)
plt.title("SHAP Summary - Random Forest (Extended)")
plt.tight_layout()
plt.savefig("cleaned_data/final/imgs/shap_summary_extended.png", bbox_inches='tight')
plt.close()

# SHAP summary plots for each class
for i, label in enumerate(labels):
    if shap_values.shape[1] > i:  # Ensure we have enough classes
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values[:, :, i], X_test_sample, feature_names=feature_names, 
                         max_display=10, show=False)
        plt.title(f"SHAP Summary - Features influencing {label} prediction")
        plt.tight_layout()
        plt.savefig(f"cleaned_data/final/imgs/shap_summary_{label.lower()}.png", bbox_inches='tight')
        plt.close()
        print(f"Generated SHAP analysis plot for {label} class")

# Model comparison summary
print("\n\nModel Comparison Summary (Weighted F1, Precision, Recall):")
for model_name, metrics in model_scores.items():
    print(f"{model_name}: F1={metrics['f1-score']:.2f}, Precision={metrics['precision']:.2f}, Recall={metrics['recall']:.2f}")

# 6. Visualize model comparison (new addition)
plt.figure(figsize=(10, 6))
metrics = ['f1-score', 'precision', 'recall']
model_names = list(model_scores.keys())
x = np.arange(len(model_names))
width = 0.2
multiplier = 0

for metric in metrics:
    metric_values = [model_scores[model][metric] for model in model_names]
    offset = width * multiplier
    plt.bar(x + offset, metric_values, width, label=metric)
    multiplier += 1

plt.xlabel('Models')
plt.ylabel('Score')
plt.title('Model Performance Comparison')
plt.xticks(x + width, model_names)
plt.legend(loc='upper left')
plt.tight_layout()
plt.savefig("cleaned_data/final/imgs/model_comparison.png")
plt.close()
print("Generated model comparison chart")

# KMeans clustering analysis (original part)
print("Performing KMeans clustering analysis...")
cluster_cols = ['victim_profession', 'department', 'violence_type']
cluster_df = df.dropna(subset=cluster_cols)
X_cluster = pd.get_dummies(cluster_df[cluster_cols])

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_cluster)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_cluster)

pca_df = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
pca_df["Cluster"] = clusters

plt.figure(figsize=(8, 6))
sns.scatterplot(data=pca_df, x="PC1", y="PC2", hue="Cluster", palette="Set2")
plt.title("KMeans Clustering of Roles & Violence Types (PCA 2D)")
plt.tight_layout()
plt.savefig("cleaned_data/final/imgs/kmeans_pca_roles.png")
plt.close()

# Cluster summaries
cluster_df['Cluster'] = clusters
for col in cluster_cols:
    print(f"\nTop 3 '{col}' values per cluster:")
    print(cluster_df.groupby('Cluster')[col].value_counts().groupby(level=0).nlargest(3))

# 7. Time feature analysis summary (new addition)
print("\n\nTime Feature Analysis:")
print("=======================")
if 'hour' in time_features.columns and valid_dates.any():
    # Analyze severity by time of day
    hour_severity = {}
    valid_hours = df.loc[valid_dates, 'event_time'].dt.hour
    valid_severity = df.loc[valid_dates, target]
    
    for hour in range(24):
        hour_data = valid_severity[valid_hours == hour]
        if not hour_data.empty:
            severity_counts = hour_data.value_counts(normalize=True)
            hour_severity[hour] = severity_counts.to_dict()
    
    # Print time-related insights
    print("Hours with highest severe incident rates:")
    severe_rates = {hour: data.get('Severe', 0) for hour, data in hour_severity.items() if 'Severe' in data}
    for hour, rate in sorted(severe_rates.items(), key=lambda x: x[1], reverse=True)[:5]:
        print(f"Hour {hour}: {rate:.2%}")
    
    print("\nWeekday vs. Weekend incident severity comparison:")
    weekday_data = valid_severity[df.loc[valid_dates, 'event_time'].dt.dayofweek < 5]
    weekend_data = valid_severity[df.loc[valid_dates, 'event_time'].dt.dayofweek >= 5]
    
    print("Weekday severity distribution:")
    print(weekday_data.value_counts(normalize=True))
    
    print("\nWeekend severity distribution:")
    print(weekend_data.value_counts(normalize=True))
else:
    print("Time features could not be extracted from the dataset.")

print("\nAnalysis complete! All charts and results have been saved to the 'cleaned_data/final/imgs/' directory.")

Processing time features...
Successfully extracted time features for 1323 records


Performing Extended EDA Analysis...
Analyzing violence patterns by shift and profession...
Analyzing relationship between violence type and emotional impact...
Generating text embedding visualization...
Analyzing departmental patterns by time...
Analyzing violence patterns by time of day...
Extended EDA analysis complete!
Preprocessing data...
Target classes: ['Mild' 'Moderate' 'Severe']
TF-IDF extracted 300 text features
Combining features...
Added time features to the model
Final feature matrix shape: (1141, 740)
Training set shape after SMOTE: (1914, 740)
Training and evaluating models...

Training Logistic Regression model...
Logistic Regression - Cross-Validation Score (F1 weighted): 0.9126 ± 0.0221

Logistic Regression - Classification Report
              precision    recall  f1-score   support

        Mild       0.50      0.41      0.45        17
    Moderate       0.83      0.82      0.82     

<Figure size 1000x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>

<Figure size 800x600 with 0 Axes>